# Test Your Trained MNIST CNN Model

This notebook loads and tests your trained model locally.

Run all cells to test your model on the MNIST test set.


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import sys
import os

sys.path.append('src')
from model import MNISTCNN

import numpy as np
import matplotlib.pyplot as plt

print("=" * 60)
print("🖥️  GPU DETECTION")
print("=" * 60)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU FOUND! Using: {device}")
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print("\n🚀 Testing will run on GPU - Fast inference expected!")
    USE_GPU = True
else:
    device = torch.device('cpu')
    print("⚠️  No GPU detected. Using: CPU")
    print("   Testing will run on CPU (slower but still works)")
    USE_GPU = False

print("=" * 60)
print(f"✅ Imports successful!")
print(f"PyTorch version: {torch.__version__}")


## 1. Load the Trained Model


In [ ]:
model_path = 'models/mnist_cnn_model.pth'

if not os.path.exists(model_path):
    print(f"❌ Model not found at {model_path}")
    print("\n📝 Make sure you've downloaded the model from Kaggle!")
    print("   Place it in: models/mnist_cnn_model.pth")
else:
    print(f"✅ Model file found at: {model_path}")
    model = MNISTCNN()
    
    if USE_GPU:
        model.load_state_dict(torch.load(model_path, map_location='cuda'))
        model = model.to(device)
        print(f"✅ Model loaded on GPU: {device}")
    else:
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        print("✅ Model loaded on CPU")
    
    model.eval()
    print("✅ Model ready for inference!")
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"📊 Model parameters: {total_params:,}")


## 2. Load Test Dataset


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_dataset = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=transform
)

print(f"✅ Test dataset loaded: {len(test_dataset)} samples")


## 3. Test on Random Samples


In [ ]:
def predict(model, image_tensor, device):
    image_tensor = image_tensor.to(device)
    with torch.no_grad():
        output = model(image_tensor.unsqueeze(0))
        probabilities = F.softmax(output, dim=1)
        predicted = torch.argmax(output, dim=1)
        confidence = probabilities[0][predicted].item() * 100
    return predicted.item(), confidence

num_samples = 10
indices = np.random.choice(len(test_dataset), num_samples, replace=False)

print(f"Testing on {num_samples} random samples:")
if USE_GPU:
    print(f"🚀 Using GPU: {device}")
else:
    print(f"⚠️  Using CPU")
print("=" * 60)

correct = 0
for i, idx in enumerate(indices):
    image, true_label = test_dataset[idx]
    predicted, confidence = predict(model, image, device)
    is_correct = predicted == true_label
    if is_correct:
        correct += 1
    
    status = "✓" if is_correct else "✗"
    print(f"Sample {i+1}: True={true_label}, Predicted={predicted}, "
          f"Confidence={confidence:.2f}%, {status}")

accuracy = 100 * correct / num_samples
print("=" * 60)
print(f"Accuracy on {num_samples} samples: {accuracy:.2f}%")


## 4. Visualize Predictions


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Model Predictions on Test Samples', fontsize=14)

for i, idx in enumerate(indices):
    row = i // 5
    col = i % 5
    ax = axes[row, col]
    
    image, true_label = test_dataset[idx]
    predicted, confidence = predict(model, image, device)
    
    image_display = image.squeeze().cpu().numpy()
    ax.imshow(image_display, cmap='gray')
    
    color = 'green' if predicted == true_label else 'red'
    ax.set_title(f'True: {true_label}\nPred: {predicted} ({confidence:.1f}%)', 
                 color=color, fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()


## 5. Calculate Overall Test Accuracy


In [ ]:
from torch.utils.data import DataLoader
import time

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
correct = 0
total = 0

print("Calculating accuracy on full test set...")
if USE_GPU:
    print(f"🚀 Using GPU: {device} - Fast inference expected!")
else:
    print("⚠️  Using CPU - This may take a minute...")

start_time = time.time()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

elapsed_time = time.time() - start_time
accuracy = 100 * correct / total

print("=" * 60)
print(f"Overall Test Accuracy: {accuracy:.2f}%")
print(f"Correct: {correct}/{total}")
print(f"Time taken: {elapsed_time:.2f} seconds")
if USE_GPU:
    print(f"🚀 GPU acceleration active!")
print("=" * 60)
